In [1]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from lib.svx001 import create_position, read_factors, scale_factors

In [3]:
from lib.cux002 import StrategyEvaluate1

In [4]:
method = 'cicso0'
instruments = 'ims'
task_id = '200037'
period = 5
name = 'lgbm'

In [5]:
strategy_settings = {'commission': 0.000024, 'slippage': 0, 'size': 200}

In [6]:
predict_data = read_factors(method=method,
                                instruments=instruments,
                                task_id=task_id,
                                period=period,
                                name=name)
predict_data.head()

,trade_time,code,nxt1_ret_5h,predict
0,2024-12-05 10:20:00,IM,-0.001860,-0.000025
5,2024-12-05 10:25:00,IM,-0.000206,-0.000011
10,2024-12-05 10:30:00,IM,0.003050,-0.000017
15,2024-12-05 10:35:00,IM,0.004669,-0.000008
20,2024-12-05 10:40:00,IM,0.000327,-0.000025


In [7]:
scale_factors(predict_data,
                  method='roll_zscore',
                  win=240,
                  factor_name='predict')

In [8]:
signal_method = 'rollrank_signal'
signal_params = {'roll_num':24, 'threshold':0.7}
strategy_method = None#'trailing_atr_strategy'
strategy_params = None#{'atr_period':14,'atr_multiplier':6, 'max_volume':1}
#signal_method = 'quantile_signal'
#signal_params = {'roll_num':20, 'threshold':0.7}

In [16]:
pos_data, total_data2 = create_position(predict_data=predict_data,
                    signal_method=signal_method,
                    signal_params=signal_params,
                    strategy_method=strategy_method,
                    strategy_params=strategy_params)

In [17]:
pos_data = pos_data.stack()
pos_data.name = 'signal'
total_data2 = total_data2.stack()

new_data = pd.concat([total_data2,pos_data],axis=1).reset_index()

In [18]:
new_data

,trade_time,code,nxt1_ret_5h,predict,transformed,signal
0,2024-12-05 10:20:00,IM,-0.001860,-0.000025,NaN,0
1,2024-12-05 10:25:00,IM,-0.000206,-0.000011,NaN,0
2,2024-12-05 10:30:00,IM,0.003050,-0.000017,NaN,0
3,2024-12-05 10:35:00,IM,0.004669,-0.000008,NaN,0
4,2024-12-05 10:40:00,IM,0.000327,-0.000025,NaN,0
...,...,...,...,...,...,...
3010,2025-03-12 14:40:00,IM,-0.001396,-0.000022,-0.143658,-1
3011,2025-03-12 14:45:00,IM,-0.002331,-0.000004,0.436705,1
3012,2025-03-12 14:50:00,IM,0.000454,-0.000015,0.086506,-1
3013,2025-03-12 14:55:00,IM,-0.000580,-0.000020,-0.058690,-1


In [19]:
from lib.cux001 import FactorEvaluate1

In [ ]:
evaluate1 = FactorEvaluate1(factor_data=predict_data,
                                factor_name='signal',
                                ret_name='nxt1_ret_{0}h'.format(period),
                                fee=0.000024,
                                scale_method='raw',
                                expression=name)